# Goal

### Generating Dungeons And Dragons Backstory

Markov Chain relies heavily on the dataset it was trained on without understanding the context, thus is perfect to generate fantasy/artificial backstory. Markov Chain model can generate backstories for DND characters faster than other Deep Learning models as it will just bluff up stuff using the provided tokens.

# Requirement

### The Dataset

Generated using:
https://capitalizemytitle.com/character-generator/dnd-backstory/

The parameters as follows:

| Parameter         | Value           |
|-------------------|-----------------|
| Gender            | any             |
| Character Name    | X               |
| Character Details | (not specified) |
| DnD Class         | any             |
| DnD Race          | any             |

The generator was run 10 times with each generation results in 3 paragraphs (3 lines), with total of 30 lines.
It is then combined in [dataset/backstory-combined.txt](dataset/backstory-combined.txt).


### Importing the package

In [1]:
import markovify

### Reading the train dataset

In [2]:
with open("dataset/backstory-combined.txt") as train_txt:
    train_data = train_txt.read()

train_data

'X, a tiefling with striking crimson skin and curling obsidian horns, was born during a violent storm in the town of Eldershade. His mother, a renowned herbalist named Liora, claimed that the tempest heralded his arrival as both an omen and a blessing. From an early age, X felt destined for more than the serene life of gathering herbs and tending to the local grove where she practiced her craft; he longed for adventure beyond the tranquil valley that sheltered him. As whispers of dark creatures creeping closer from the Shadow Vale reached their ears, Liora\'s warnings echoed in his mind: \\"The world outside is not kind to those who wield power.\\" But drawn by tales of heroism and glory spun by passing travelers at local taverns, X couldn\'t resist planting one foot on home soil while stretching another toward destiny.\nWhen Liora vanished one moonless night without a traceâ€”taken by dark forces known only in folkloreâ€”X\'s plans shifted drastically from idyllic exploration to venge

### Manual preprocessing

In [3]:
# train_data = train_data.replace("\\n", "")
# train_data = train_data.replace("\\'", "")
#
# train_data

### Replacing the X (character name context)

In [4]:
CHARACTER_NAME = "Circle Lovelace"

train_data = train_data.replace('X', CHARACTER_NAME)

### Building the model

In [5]:
# Number of words a next word depends on
state_size = 3

# Randomness chooser
# - TEMPERATURE == 1    : next word is chosen according to the observed state probabilities
# - TEMPERATURE < 1     : the higher a word's probability is, it's more likely to be used
# - TEMPERATURE > 1     : the lesser a word's probability is, it's more likely to be used, increases randomness
# DEFAULT TEMPERATURE IS 1.0
markovify.config.TEMPERATURE = 0.7

markov_model = markovify.Text(
    input_text=train_data,
    state_size=state_size
)

markov_model.state_size

3

In [6]:
markov_model.chain.model

{('___BEGIN__', '___BEGIN__', '___BEGIN__'): {'Circle': 7,
  'His': 4,
  'From': 3,
  'He': 1,
  'The': 4,
  'Cursed': 1,
  'Determined': 1,
  'Alongside': 1,
  'Yet,': 1,
  'As': 8,
  'Born': 1,
  'While': 1,
  'However,': 1,
  'Fueled': 1,
  'Yet': 2,
  'In': 1,
  'But': 1,
  'Wandering': 1,
  'This': 1,
  'With': 1,
  'Raised': 1,
  'They': 1,
  'It': 4,
  'During': 1,
  'Her': 1,
  'Years': 1,
  'Abandoned': 1,
  'One': 1,
  'Despite': 1,
  'An': 1,
  'Little': 1,
  'Under': 1,
  'When': 1,
  'Now': 1,
  'Each': 1},
 ('___BEGIN__', '___BEGIN__', 'Circle'): {'Lovelace,': 3, 'Lovelace': 4},
 ('___BEGIN__', 'Circle', 'Lovelace,'): {'a': 3},
 ('Circle', 'Lovelace,', 'a'): {'tiefling': 1, 'name': 1, 'half-elf': 1},
 ('Lovelace,', 'a', 'tiefling'): {'with': 1},
 ('a', 'tiefling', 'with'): {'striking': 1},
 ('tiefling', 'with', 'striking'): {'crimson': 1},
 ('with', 'striking', 'crimson'): {'skin': 1},
 ('striking', 'crimson', 'skin'): {'and': 1},
 ('crimson', 'skin', 'and'): {'curling': 

In [7]:
# # Let's see the chaining result
#
# target_state = ("Circle")
#
# frequencies = {}
#
# for key, val in zip(markov_model.chain.model[target_state][0], markov_model.chain.model[target_state][1]):
#     frequencies[key] = val
#
# frequencies

In [8]:
# (Optional) Compile to improve the model's generation speed and reduce its size
markov_model = markov_model.compile()

### Generating Short Sentences

In [9]:
# Generates a sentence no more than 300 characters
result = markov_model.make_short_sentence(300)

print(result)

It was during one such adventure when Circle Lovelace stumbled upon a tattered tome sealed away behind crumbling stone.


### Generating a paragraph with N character limits

In [10]:
# Generation parameters
N_SENTENCES = 5
N_CHARACTER_LIMIT = 300

results = []
for _ in range(N_SENTENCES):
    results.append(markov_model.make_short_sentence(N_CHARACTER_LIMIT))

print(" ".join(results))

Raised in a quaint village nestled against the Mistwood Forest, Circle Lovelace was born in the heart of Tharindal. It was during one such adventure when Circle Lovelace stumbled upon a tattered tome sealed away behind crumbling stone. The village was once a vibrant hub for those who sought to shield their son from his mystical heritage. But from a young age. Abandoned at birth by a mother who vanished into the shadows, Circle Lovelace was born in the heart of Tharindal.


In [11]:
# Generation parameters
N_SENTENCES = 5
min_words = 4
max_words = 25

results = []
for _ in range(N_SENTENCES):
    result = None
    # Prevent no result
    while result is None:
        result = markov_model.make_sentence(
            min_words=min_words,
            max_words=max_words
        )
    results.append(result)

print(" ".join(results))

Raised in a quaint village nestled against the Mistwood Forest, Circle Lovelace was born in the heart of Tharindal. Abandoned at birth by a mother who vanished into the shadows, Circle Lovelace was born in the heart of Tharindal. Abandoned at birth by a mother who vanished into the shadows, Circle Lovelace was born in the heart of Tharindal. The village was once a vibrant hub for those who sought to shield their son from his mystical heritage. But from a young age.


### Generating with prefix


In [12]:
def generate_paragraphs(
        prefix: str = None,
        n_sentences: int = 5,
        n_paragraph: int = 3,
        min_words: int = 4,
        max_words: int = 25
):
    prefix_tuple = ()
    if prefix is not None:
        prefix_tuple = tuple(prefix.split())

    paragraphs = []
    for _ in range(n_paragraph):
        paragraph = []
        for i in range(n_sentences):
            sentence = None
            # Prevent no result
            while sentence is None:
                if i != 0:
                    sentence = markov_model.make_sentence(
                        min_words=min_words,
                        max_words=max_words
                    )
                else:
                    sentence = markov_model.make_sentence(
                        init_state=None if len(prefix_tuple) == 0 else prefix_tuple,
                        min_words=min_words,
                        max_words=max_words
                    )
            paragraph.append(sentence)

        paragraphs.append(" ".join(paragraph))

    return paragraphs

paragraphs = generate_paragraphs()
print("\n\n".join(paragraphs))

Raised in a quaint village nestled against the Mistwood Forest, Circle Lovelace was born in the heart of Tharindal. Circle Lovelace was the only child of humble farmers who sought to exploit or erase such potential threats. Raised in a quaint village nestled against the Mistwood Forest, Circle Lovelace was born in the heart of Tharindal. It was during one such adventure when Circle Lovelace stumbled upon a tattered tome sealed away behind crumbling stone. The village was once a vibrant hub for those who sought to exploit or erase such potential threats.

Raised in a quaint village nestled against the Mistwood Forest, Circle Lovelace was born in the heart of Tharindal. It was during one such adventure when Circle Lovelace stumbled upon a tattered tome sealed away behind crumbling stone. It was during one such adventure when Circle Lovelace turned to forbidden texts hidden in his late grandfather's attic. From an early age, it became clear that Circle Lovelace was born in the heart of Th

### Generating Multiple Paragraphs